In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
print("Tensorflow version: {}".format(tf.version.VERSION))
print("Eager mode: {}".format(tf.executing_eagerly()))
print("Hub version: {}".format(hub.__version__))
print("GPU is {}".format("available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE"))

Tensorflow version: 2.0.0
Eager mode: True
Hub version: 0.7.0
GPU is available


In [3]:
# 完整的数据按照5:5的比例进行切割得到训练集(train)和测试集(test)
# 将训练集按照 6:4 的比例进行切割得到新的训练集(train)和验证集(validation)
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

In [6]:
(train_data, validation_data), test_data = tfds.load(
    name='imdb_reviews', # 网络电影数据库(Internet Movie Database)
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\Administrator\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.